<div align = "center">

# **Calculos Diarios**

</div>

In [18]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas correctamente")
print(f"Fecha actual: {datetime.now().strftime('%Y-%m-%d')}")

Librerías cargadas correctamente
Fecha actual: 2025-07-25


## 1. Descarga de Datos Actualizados

In [19]:
# Cargar datos desde los CSV guardados en lugar de descargar de Yahoo Finance
print("=== CARGANDO DATOS DESDE CSV ===")

# Cargar S&P 500 desde CSV
sp500_csv = pd.read_csv('../data/input/sp500_2years.csv')
sp500_csv['Date'] = pd.to_datetime(sp500_csv['Date'])

print(f"S&P 500 - Registros cargados: {len(sp500_csv)}")
print(f"Fechas: {sp500_csv['Date'].iloc[0].strftime('%Y-%m-%d')} a {sp500_csv['Date'].iloc[-1].strftime('%Y-%m-%d')}")

# Cargar VIX desde CSV  
vix_csv = pd.read_csv('../data/input/vix_2years.csv')
vix_csv['Date'] = pd.to_datetime(vix_csv['Date'])

print(f"VIX - Registros cargados: {len(vix_csv)}")
print(f"Fechas: {vix_csv['Date'].iloc[0].strftime('%Y-%m-%d')} a {vix_csv['Date'].iloc[-1].strftime('%Y-%m-%d')}")

# Verificar últimos 5 días para confirmar que tenemos la semana 21-25 julio
print(f"\n=== VERIFICACIÓN SEMANA OBJETIVO (21-25 JULIO) ===")
print("Últimos 5 días S&P 500:")
print(sp500_csv.tail()['Date'].dt.strftime('%Y-%m-%d %A').tolist())
print("\nÚltimos 5 días VIX:")
print(vix_csv.tail()['Date'].dt.strftime('%Y-%m-%d %A').tolist())

# Asignar a las variables sp500 y vix para mantener compatibilidad con el resto del código
sp500 = sp500_csv.set_index('Date')
vix = vix_csv.set_index('Date')

=== CARGANDO DATOS DESDE CSV ===
S&P 500 - Registros cargados: 502
Fechas: 2023-07-26 a 2025-07-25
VIX - Registros cargados: 502
Fechas: 2023-07-26 a 2025-07-25

=== VERIFICACIÓN SEMANA OBJETIVO (21-25 JULIO) ===
Últimos 5 días S&P 500:
['2025-07-21 Monday', '2025-07-22 Tuesday', '2025-07-23 Wednesday', '2025-07-24 Thursday', '2025-07-25 Friday']

Últimos 5 días VIX:
['2025-07-21 Monday', '2025-07-22 Tuesday', '2025-07-23 Wednesday', '2025-07-24 Thursday', '2025-07-25 Friday']


In [20]:
# Preparar datos para guardar: solo Date y Close
# Simplificar usando directamente los DataFrames y seleccionando solo las columnas necesarias

# Para S&P 500
sp500_clean = sp500[['Close']].reset_index()
sp500_clean.columns = ['Date', 'Close']

# Para VIX  
vix_clean = vix[['Close']].reset_index()
vix_clean.columns = ['Date', 'Close']

# Guardar en CSV
sp500_clean.to_csv('../data/input/sp500_2years.csv', index=False)
vix_clean.to_csv('../data/input/vix_2years.csv', index=False)

print("=== ARCHIVOS GUARDADOS ===")
print(f"S&P 500: ../data/input/sp500_2years.csv ({len(sp500_clean)} registros)")
print(f"VIX: ../data/input/vix_2years.csv ({len(vix_clean)} registros)")

# Mostrar primeros y últimos registros
print(f"\n=== VERIFICACIÓN DATOS S&P 500 ===")
print("Primeros 3 registros:")
print(sp500_clean.head(3))
print("Últimos 3 registros:")
print(sp500_clean.tail(3))

print(f"\n=== VERIFICACIÓN DATOS VIX ===") 
print("Primeros 3 registros:")
print(vix_clean.head(3))
print("Últimos 3 registros:")
print(vix_clean.tail(3))

=== ARCHIVOS GUARDADOS ===
S&P 500: ../data/input/sp500_2years.csv (502 registros)
VIX: ../data/input/vix_2years.csv (502 registros)

=== VERIFICACIÓN DATOS S&P 500 ===
Primeros 3 registros:
        Date        Close
0 2023-07-26  4566.750000
1 2023-07-27  4537.410156
2 2023-07-28  4582.229980
Últimos 3 registros:
          Date        Close
499 2025-07-23  6358.910156
500 2025-07-24  6363.350098
501 2025-07-25  6388.640137

=== VERIFICACIÓN DATOS VIX ===
Primeros 3 registros:
        Date  Close
0 2023-07-26  13.19
1 2023-07-27  14.41
2 2023-07-28  13.33
Últimos 3 registros:
          Date  Close
499 2025-07-23  15.37
500 2025-07-24  15.39
501 2025-07-25  14.93


## 2. Modelado GARCH con Ventanas Deslizantes

In [21]:
# Usar los datos ya cargados desde CSV en lugar de cargarlos nuevamente
sp500_data = sp500_csv.copy()  # Ya están cargados desde CSV
vix_data = vix_csv.copy()      # Ya están cargados desde CSV

# Calcular retornos logarítmicos del S&P 500
sp500_data['returns'] = np.log(sp500_data['Close'] / sp500_data['Close'].shift(1)) * 100

# Definir las fechas objetivo (21-25 julio 2025)
fechas_objetivo = pd.to_datetime(['2025-07-21', '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25'])

print("=== CONFIGURACIÓN VENTANAS DESLIZANTES ===")
print(f"Fechas objetivo para predicción: {fechas_objetivo.strftime('%Y-%m-%d %A').tolist()}")
print(f"Ventana de entrenamiento: 252 días de trading (≈ 1 año)")

# Función para obtener ventana de 1 año (252 días de trading) antes de una fecha
def obtener_ventana_entrenamiento(fecha_prediccion, datos, dias_trading=252):
    # Encontrar el índice de la fecha de predicción
    idx_prediccion = datos[datos['Date'] == fecha_prediccion].index[0]
    
    # Obtener ventana de 252 días de trading anteriores (excluyendo el día de predicción)
    inicio_ventana = max(0, idx_prediccion - dias_trading)
    fin_ventana = idx_prediccion
    
    ventana = datos.iloc[inicio_ventana:fin_ventana].copy()
    
    print(f"\nVentana para {fecha_prediccion.strftime('%Y-%m-%d')}:")
    print(f"  - Registros: {len(ventana)}")
    print(f"  - Desde: {ventana['Date'].iloc[0].strftime('%Y-%m-%d')}")
    print(f"  - Hasta: {ventana['Date'].iloc[-1].strftime('%Y-%m-%d')}")
    
    return ventana

# Verificar ventanas para cada fecha objetivo
print("\n=== VERIFICACIÓN DE VENTANAS ===")
for fecha in fechas_objetivo:
    if fecha <= sp500_data['Date'].max():
        ventana = obtener_ventana_entrenamiento(fecha, sp500_data)
    else:
        print(f"\n⚠️  Fecha {fecha.strftime('%Y-%m-%d')} no disponible en los datos")

=== CONFIGURACIÓN VENTANAS DESLIZANTES ===
Fechas objetivo para predicción: ['2025-07-21 Monday', '2025-07-22 Tuesday', '2025-07-23 Wednesday', '2025-07-24 Thursday', '2025-07-25 Friday']
Ventana de entrenamiento: 252 días de trading (≈ 1 año)

=== VERIFICACIÓN DE VENTANAS ===

Ventana para 2025-07-21:
  - Registros: 252
  - Desde: 2024-07-17
  - Hasta: 2025-07-18

Ventana para 2025-07-22:
  - Registros: 252
  - Desde: 2024-07-18
  - Hasta: 2025-07-21

Ventana para 2025-07-23:
  - Registros: 252
  - Desde: 2024-07-19
  - Hasta: 2025-07-22

Ventana para 2025-07-24:
  - Registros: 252
  - Desde: 2024-07-22
  - Hasta: 2025-07-23

Ventana para 2025-07-25:
  - Registros: 252
  - Desde: 2024-07-23
  - Hasta: 2025-07-24


In [22]:
# Función para ajustar modelo GARCH y hacer predicción
def ajustar_garch_y_predecir(fecha_prediccion, datos):
    # Obtener ventana de entrenamiento
    ventana = obtener_ventana_entrenamiento(fecha_prediccion, datos)
    
    # Obtener retornos sin NaN
    retornos = ventana['returns'].dropna()
    
    if len(retornos) < 50:  # Verificar que tengamos suficientes datos
        print(f"⚠️  Muy pocos datos para {fecha_prediccion.strftime('%Y-%m-%d')}")
        return None, None
    
    try:
        # Ajustar modelo GARCH(1,1) - misma configuración que notebook principal
        modelo = arch_model(
            retornos,
            vol='GARCH',
            p=1,
            q=1,
            mean='Constant',
            dist='Normal',
            rescale=True
        )
        
        resultado = modelo.fit(disp='off')
        
        # Hacer predicción para 1 día adelante
        prediccion = resultado.forecast(horizon=1)
        volatilidad_predicha = np.sqrt(prediccion.variance.values[-1, 0])
        
        print(f"  ✓ GARCH ajustado - Volatilidad predicha: {volatilidad_predicha:.4f}")
        
        return resultado, volatilidad_predicha
        
    except Exception as e:
        print(f"  ❌ Error ajustando GARCH: {e}")
        return None, None

# Diccionario para almacenar resultados
resultados_garch = {}

print("=== AJUSTANDO MODELOS GARCH ===")
for fecha in fechas_objetivo:
    if fecha <= sp500_data['Date'].max():
        print(f"\n📊 Procesando {fecha.strftime('%Y-%m-%d %A')}...")
        modelo, prediccion = ajustar_garch_y_predecir(fecha, sp500_data)
        
        if modelo is not None:
            resultados_garch[fecha] = {
                'modelo': modelo,
                'prediccion_volatilidad': prediccion,
                'fecha': fecha
            }
        else:
            resultados_garch[fecha] = {
                'modelo': None,
                'prediccion_volatilidad': None,
                'fecha': fecha
            }
    else:
        print(f"\n⚠️  Saltando {fecha.strftime('%Y-%m-%d')} - no disponible")

print(f"\n=== RESUMEN MODELOS AJUSTADOS ===")
print(f"Total fechas procesadas: {len([k for k in resultados_garch.keys() if resultados_garch[k]['modelo'] is not None])}")
for fecha, resultado in resultados_garch.items():
    if resultado['modelo'] is not None:
        print(f"  ✓ {fecha.strftime('%Y-%m-%d')}: Predicción = {resultado['prediccion_volatilidad']:.4f}")
    else:
        print(f"  ❌ {fecha.strftime('%Y-%m-%d')}: Falló el ajuste")

=== AJUSTANDO MODELOS GARCH ===

📊 Procesando 2025-07-21 Monday...

Ventana para 2025-07-21:
  - Registros: 252
  - Desde: 2024-07-17
  - Hasta: 2025-07-18
  ✓ GARCH ajustado - Volatilidad predicha: 0.7021

📊 Procesando 2025-07-22 Tuesday...

Ventana para 2025-07-22:
  - Registros: 252
  - Desde: 2024-07-18
  - Hasta: 2025-07-21
  ✓ GARCH ajustado - Volatilidad predicha: 0.6812

📊 Procesando 2025-07-23 Wednesday...

Ventana para 2025-07-23:
  - Registros: 252
  - Desde: 2024-07-19
  - Hasta: 2025-07-22
  ✓ GARCH ajustado - Volatilidad predicha: 0.6597

📊 Procesando 2025-07-24 Thursday...

Ventana para 2025-07-24:
  - Registros: 252
  - Desde: 2024-07-22
  - Hasta: 2025-07-23
  ✓ GARCH ajustado - Volatilidad predicha: 0.6996

📊 Procesando 2025-07-25 Friday...

Ventana para 2025-07-25:
  - Registros: 252
  - Desde: 2024-07-23
  - Hasta: 2025-07-24
  ✓ GARCH ajustado - Volatilidad predicha: 0.6749

=== RESUMEN MODELOS AJUSTADOS ===
Total fechas procesadas: 5
  ✓ 2025-07-21: Predicción = 0

## 3. Cálculo de Volatilidad Real y VIX

In [23]:
# Calcular volatilidad histórica real (ventana de 7 días) - misma metodología que notebook
sp500_data['volatilidad_historica'] = sp500_data['returns'].rolling(window=7).std()

# Función para obtener volatilidad real y VIX para una fecha específica
def obtener_datos_reales(fecha, sp500_data, vix_data):
    # Volatilidad real calculada
    sp500_row = sp500_data[sp500_data['Date'] == fecha]
    vix_row = vix_data[vix_data['Date'] == fecha]
    
    if len(sp500_row) == 0 or len(vix_row) == 0:
        return None, None
    
    volatilidad_real = sp500_row['volatilidad_historica'].iloc[0]
    vix_valor = vix_row['Close'].iloc[0] / 100  # Convertir a escala decimal
    
    return volatilidad_real, vix_valor

# Diccionario para almacenar datos reales
datos_reales = {}

print("=== CALCULANDO DATOS REALES ===")
for fecha in fechas_objetivo:
    print(f"\n📈 {fecha.strftime('%Y-%m-%d %A')}:")
    
    vol_real, vix_real = obtener_datos_reales(fecha, sp500_data, vix_data)
    
    if vol_real is not None and vix_real is not None:
        datos_reales[fecha] = {
            'volatilidad_real': vol_real,
            'vix': vix_real,
            'fecha': fecha
        }
        print(f"  ✓ Volatilidad real: {vol_real:.4f}")
        print(f"  ✓ VIX (decimal): {vix_real:.4f}")
    else:
        datos_reales[fecha] = {
            'volatilidad_real': np.nan,
            'vix': np.nan,
            'fecha': fecha
        }
        print(f"  ❌ Datos no disponibles")

print(f"\n=== RESUMEN DATOS REALES ===")
for fecha, datos in datos_reales.items():
    if not np.isnan(datos['volatilidad_real']):
        print(f"  ✓ {fecha.strftime('%Y-%m-%d')}: Vol Real = {datos['volatilidad_real']:.4f}, VIX = {datos['vix']:.4f}")
    else:
        print(f"  ❌ {fecha.strftime('%Y-%m-%d')}: Sin datos")

=== CALCULANDO DATOS REALES ===

📈 2025-07-21 Monday:
  ✓ Volatilidad real: 0.3346
  ✓ VIX (decimal): 0.1665

📈 2025-07-22 Tuesday:
  ✓ Volatilidad real: 0.2885
  ✓ VIX (decimal): 0.1650

📈 2025-07-23 Wednesday:
  ✓ Volatilidad real: 0.3836
  ✓ VIX (decimal): 0.1537

📈 2025-07-24 Thursday:
  ✓ Volatilidad real: 0.2912
  ✓ VIX (decimal): 0.1539

📈 2025-07-25 Friday:
  ✓ Volatilidad real: 0.2948
  ✓ VIX (decimal): 0.1493

=== RESUMEN DATOS REALES ===
  ✓ 2025-07-21: Vol Real = 0.3346, VIX = 0.1665
  ✓ 2025-07-22: Vol Real = 0.2885, VIX = 0.1650
  ✓ 2025-07-23: Vol Real = 0.3836, VIX = 0.1537
  ✓ 2025-07-24: Vol Real = 0.2912, VIX = 0.1539
  ✓ 2025-07-25: Vol Real = 0.2948, VIX = 0.1493


## 4. Tabla Comparativa Final

In [24]:
# Crear tabla comparativa combinando todos los resultados
tabla_comparativa = []

print("=== CONSTRUYENDO TABLA COMPARATIVA ===")
for fecha in fechas_objetivo:
    # Obtener datos GARCH
    garch_pred = resultados_garch.get(fecha, {}).get('prediccion_volatilidad', np.nan)
    
    # Obtener datos reales
    vol_real = datos_reales.get(fecha, {}).get('volatilidad_real', np.nan)
    vix_valor = datos_reales.get(fecha, {}).get('vix', np.nan)
    
    # Calcular errores
    error_garch = np.abs(garch_pred - vol_real) if not (np.isnan(garch_pred) or np.isnan(vol_real)) else np.nan
    error_vix = np.abs(vix_valor - vol_real) if not (np.isnan(vix_valor) or np.isnan(vol_real)) else np.nan
    
    # Agregar fila a la tabla
    fila = {
        'Fecha': fecha.strftime('%Y-%m-%d'),
        'Día': fecha.strftime('%A'),
        'Volatilidad_Real': vol_real,
        'Predicción_GARCH': garch_pred,
        'VIX': vix_valor,
        'Error_GARCH': error_garch,
        'Error_VIX': error_vix
    }
    
    tabla_comparativa.append(fila)

# Convertir a DataFrame
df_comparativa = pd.DataFrame(tabla_comparativa)

# Mostrar tabla con formato
print("\n" + "="*100)
print("TABLA COMPARATIVA: PREDICCIONES vs REALIDAD")
print("="*100)

# Formatear para mostrar
df_display = df_comparativa.copy()
for col in ['Volatilidad_Real', 'Predicción_GARCH', 'VIX', 'Error_GARCH', 'Error_VIX']:
    df_display[col] = df_display[col].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(df_display.to_string(index=False))

# Estadísticas resumen
print("\n" + "="*60)
print("ESTADÍSTICAS RESUMEN")
print("="*60)

# Filtrar solo datos válidos
df_validos = df_comparativa.dropna(subset=['Volatilidad_Real', 'Predicción_GARCH', 'VIX'])

if len(df_validos) > 0:
    # MAE (Mean Absolute Error)
    mae_garch = df_validos['Error_GARCH'].mean()
    mae_vix = df_validos['Error_VIX'].mean()
    
    # RMSE (Root Mean Square Error)  
    rmse_garch = np.sqrt((df_validos['Error_GARCH']**2).mean())
    rmse_vix = np.sqrt((df_validos['Error_VIX']**2).mean())
    
    # Correlaciones
    corr_garch = df_validos['Volatilidad_Real'].corr(df_validos['Predicción_GARCH'])
    corr_vix = df_validos['Volatilidad_Real'].corr(df_validos['VIX'])
    
    print(f"Días con datos válidos: {len(df_validos)}")
    print(f"\nMAE (Error Absoluto Medio):")
    print(f"  - GARCH: {mae_garch:.4f}")
    print(f"  - VIX:   {mae_vix:.4f}")
    print(f"\nRMSE (Error Cuadrático Medio):")
    print(f"  - GARCH: {rmse_garch:.4f}")
    print(f"  - VIX:   {rmse_vix:.4f}")
    print(f"\nCorrelación con Volatilidad Real:")
    print(f"  - GARCH: {corr_garch:.4f}")
    print(f"  - VIX:   {corr_vix:.4f}")
    
    # Determinar mejor modelo
    mejor_mae = "GARCH" if mae_garch < mae_vix else "VIX"
    mejor_rmse = "GARCH" if rmse_garch < rmse_vix else "VIX"
    mejor_corr = "GARCH" if corr_garch > corr_vix else "VIX"
    
    print(f"\n{'='*30}")
    print("MEJOR MODELO POR MÉTRICA:")
    print(f"{'='*30}")
    print(f"MAE:         {mejor_mae}")
    print(f"RMSE:        {mejor_rmse}")
    print(f"Correlación: {mejor_corr}")
    
else:
    print("⚠️  No hay suficientes datos válidos para calcular estadísticas")

# Guardar tabla en CSV
df_comparativa.to_csv('../data/processed/comparacion_predicciones_diarias.csv', index=False)
print(f"\n📁 Tabla guardada en: ../data/processed/comparacion_predicciones_diarias.csv")

=== CONSTRUYENDO TABLA COMPARATIVA ===

TABLA COMPARATIVA: PREDICCIONES vs REALIDAD
     Fecha       Día Volatilidad_Real Predicción_GARCH    VIX Error_GARCH Error_VIX
2025-07-21    Monday           0.3346           0.7021 0.1665      0.3675    0.1681
2025-07-22   Tuesday           0.2885           0.6812 0.1650      0.3927    0.1235
2025-07-23 Wednesday           0.3836           0.6597 0.1537      0.2762    0.2299
2025-07-24  Thursday           0.2912           0.6996 0.1539      0.4084    0.1373
2025-07-25    Friday           0.2948           0.6749 0.1493      0.3801    0.1455

ESTADÍSTICAS RESUMEN
Días con datos válidos: 5

MAE (Error Absoluto Medio):
  - GARCH: 0.3650
  - VIX:   0.1609

RMSE (Error Cuadrático Medio):
  - GARCH: 0.3679
  - VIX:   0.1652

Correlación con Volatilidad Real:
  - GARCH: -0.4882
  - VIX:   -0.0275

MEJOR MODELO POR MÉTRICA:
MAE:         VIX
RMSE:        VIX
Correlación: VIX

📁 Tabla guardada en: ../data/processed/comparacion_predicciones_diarias.csv
